In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

Copyright **Paolo Raiteri**, January 2022

# Numerical solution of equilibrium problems

Let's consider a generic chemical reaction 

\begin{equation}
\nu_aA + \nu_bB + \dots \leftrightharpoons \nu_xX + \nu_yY + \dots \tag{1}
\end{equation}

if the concentrations of the species are not the equilibrium ones, the reaction will progress (either forward or backwards) to reach the equilibrium conditions, and the free energy of the system will decrease.

An infinitesimal change of the system's free energy can then be written in terms of the chemical potential, $\mu$, of all the species involved in the reaction.

\begin{equation}
\mathrm{d}G = \sum_i \mu_i \mathrm{d}n_i \tag{2}
\end{equation}

Because the concentrations of reactants and products are coupled through the reaction equation, they cannot change independently, _i.e._ if any products are formed, some reactants must be consumed.
We can therefore define a unique parameter, $\xi$, as the _progress of the reaction_ 

\begin{equation}
\mathrm{d}\xi = \frac{\mathrm{d}n_i}{ \nu_i } \tag{3}
\end{equation}

where $\nu_i$ is the stoichiometric coefficient of the species $i$ taken as a positive number for products (formed) and as a negative number for reactants (consumed), and $\mathrm{d}n_i$ is an infinitesimal change in the concentration of species $i$.
Thus, the free energy change due to the progress of the reaction (in any direction) becomes

\begin{equation}
\mathrm{d}G = \sum_i \nu_i \mu_i \mathrm{d}\xi \tag{4}
\end{equation}

At equilibrium, the concentrations of all species are constant, $\mathrm{d}n_i=0$, and the reaction is not _progressing_ anymore, $\mathrm{d}\xi=0$, hence the free energy is not changing anymore.
On the other hand, if the system is out of equilibrium, there is a driving force that pulls the system towards equilibrium. 
A force can always be obtained as the negative of the derivative of the energy with respect to some coordinate.
In this case, the driving force that pushes the reaction towards equilibrium can be immediately obtained from equation (4).

\begin{equation}
\frac{\mathrm{d}G}{\mathrm{d}\xi} = \sum_i \nu_i \mu_i = -force \tag{5}
\end{equation}

If we then substitute the definition of the chemical potential, $\mu=\mu^0+RT\ln x$, where $x$ is the molar fraction of the species, we obtain

\begin{eqnarray}
\frac{\mathrm{d} G}{\mathrm{d}\xi} &=& \sum_i\nu_i\mu_i\\ 
&=& \sum_i\nu_i \big[\mu_i^0 +RT\ln x_i\big]\\
&=& \sum_i\nu_i\mu_i^0 +RT\sum_i\nu_i\ln x_i\\
&=& \Delta_r G^0 +RT\sum_i\ln x_i^{\nu_i}\\
&=& \Delta_r G^0 +RT\ln \prod_i x_i^{\nu_i}\\
&=& \Delta_r G^0 +RT\ln Q
\end{eqnarray}

here $Q$ is the reaction quotient. By remembering the definition of equilibrium constant, $-RT\ln K_{eq}=\Delta_r G^0$ we get

\begin{equation}
force = -\frac{\mathrm{d} G}{\mathrm{d}\xi} = RT\ln K_{eq} - RT\ln Q = RT\ln\ \frac{K_{eq}}{Q} \tag{6}
\end{equation}

This equation provides us with a conceptual framework to solve *numerically* any equilibrium problem.
Because the above equations are differential, they are valid only for infinitesimal changes in the conditions.
Hence each a small change in the concentrations, $\nu_i\delta c$ the driving force may change significantly.

Therefore we need to set up an iterative procedure where we compute the driving force, change the concentrations, compute the driving force\dots until the equilibrium is reached.

Given the concentration of all species, we can compute the driving force using the equation above and alter the concentrations of all species using

\begin{equation}
[C]_i^1 = [C]_i^0 + \mathrm{d}n_i \tag{7}
\end{equation}

where the superscript indicates a the passage of an infinitesimal amount of time, and $\delta n_i$ is

\begin{equation}
\mathrm{d}n_i = \nu_i \times force \times \delta c  \tag{7}
\end{equation}

where $\nu_i$ are the stoichiometric coefficients of the species and $\mathrm{d}c$ is an adjustable parameter.
$\mathrm{d}c$ must be small enough to maintain the validity of the approximation abover but not too small to allow for a quick convergence of the procedure.

We cen then compute the new reaction quotient, the new driving force and update the concentrations again. This procedure can then be repeated a number of times, until the force is negligible, which indicates that the calculation has converged.


## Example #1: Dimerisation reaction
In order to elucidate how that procedure works we can take a model dimerisation reaction

\begin{equation}
2A \leftrightharpoons B
\end{equation}

whose equilibrium constant can be written as
\begin{equation}
K_{eq} = \frac{[B]}{[A]^2}
\end{equation}

We now want to calculate the equilibrium concentrations of $[A]_{eq}$ and $[B]_{eq}$ given their initial concentrations $[A]_{0}$ and $[B]_{0}$.
Although this is a simple problem that can be solved analytically, in this workshop we will learn how we can use an iterative method to numerically solve it.
We will use a relatively simple minimisation procedure that can be applied to a large number of problems, for which it is not possible or it is too complicated to get an analytic solution.

Imagine mixing the reagents and then to be able to monitor the concentration of all the species in the system at very short discrete time intervals (*timesteps*). What you will see is that the concentrations will change and tend to the equilibrium value. As you have learnt in first year, the reaction quotient, $Q$, can be used to decided which way to reaction will proceed, and that at equilibrium the reaction quotient is equal to the equilibrium constant. Hence, as we have discussed in class, the reaction quotient and the equilibrium constant can be use to define a *driving force* that pulls the system towards equilibrium.

This *driving force* can then be used in conjunction with an *ICE* table to numerically compute the equilibrium concentration of reactant and products.


The working principle of the minimisation procedure that we will employ is

1. compute the reaction quotient
\begin{equation}
Q = \frac{\mathrm{[B]}_0}{\mathrm{[A]}^2_0}
\end{equation}

2. compute the driving force

\begin{equation}
F = RT \ln\bigg[\frac{K_{eq}}{Q}\bigg]
\end{equation}

3. evolve the concentrations using the ICE table

|    <img width=50/>  | [A]  <img width=150/>      | [B]<img width=150/>
| :---: | :--------: |:---------:
| *I*  | [A]$_0$    | [B]$_0$
| *C*  | $-2F\delta c$        | $+F\delta c$
| *E*  | [A]$_0$-2$F\delta c$ | [B]$_0$+$F\delta c$

4. Repeat until the solution does not change 

Let's try to implement this

# Working notebooks

In [2]:
import ipywidgets as ipw
import os

from IPython.display import Javascript
import glob as glob
import nbformat as nbf

label_layout = ipw.Layout(width='300px')

##########
pfiles = ['.protectedFiles.txt' , '../.protectedFiles.txt']
for fff in pfiles:
    if os.path.isfile(fff):
        with open(fff) as f:
            protectedFiles = f.read().splitlines()
##########
def launchNotebook(filename):
    text = "   var name_of_the_notebook = '" + filename + "'"
    vv="""
    var url = window.location.href.split('/')
    var newurl = url[0] + '//'
    for (var i = 1; i < url.length - 1; i++) {
        console.log(url[i], newurl)
        newurl += url[i] + '/'
    }
    newurl += name_of_the_notebook
    window.open(newurl)
    """
    text = text + vv
    display(Javascript(text))

def openNewNotebook(btn):
    if os.path.exists(notebookeName.value):
        print("Filename exists - Please select a different name")
        return
    
    nb = nbf.v4.new_notebook()
    text = """# Click 'Edit App' to start coding"""

    code = """\
# python packages
import pandas as pd # Dataframes and reading CSV files
import numpy as np # Numerical libraries
import matplotlib.pyplot as plt # Plotting library
from lmfit import Model # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library"""

    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                   nbf.v4.new_code_cell(code)]

    if notebookeName.value in protectedFiles or notebookeName.value in listOfFiles:
        print("File already exists, select a different filename")
    else:
        with open(notebookeName.value, 'w') as f:
            nbf.write(nb, f)
        launchNotebook(notebookeName.value)

##########
listOfFiles = []
files = glob.glob1("./","*.ipynb")
for f in files:
    if f in protectedFiles:
        continue
    listOfFiles.append(f)

def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            oldNotebookeName[0] = listOfFiles[i]

def createMenuFiles(data):
    option_list = ["Choose one"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="", options=option_list, layout=ipw.Layout(width="300px"))
    dropdown.observe(dropdown_filesHandler, names='value')

    return dropdown

##########
oldNotebookeName = ["None"]
def openOldNotebook(btn):
    if oldNotebookeName[0] == "None":
        print("Please select a filename")
    elif oldNotebookeName[0] in protectedFiles:
        print("Please select a different filename")
    else:
        launchNotebook(oldNotebookeName[0])

##########
actions0 = []

notebookeName = ipw.Text("Empty.ipynb")

btn_new = ipw.Button(description="Create a new notebook", layout=label_layout)
btn_new.on_click(openNewNotebook)

btn_old = ipw.Button(description="Open an old notebook", layout=label_layout)
btn_old.on_click(openOldNotebook)

actions0.append(ipw.HBox([btn_new,notebookeName]))
actions0.append(ipw.HBox([btn_old,createMenuFiles(listOfFiles)]))
ipw.VBox(actions0)

## Critical thinking questions

1. Verify that the final equilibrium concentrations are independent of the starting conditions, provided that the the mass is conserved. Can you explain why is that?
2. What is the effect of changing the values of $\delta c$ or the number of cycles?
3. Is the value of RT important?
4. In some cases, it is possible that the concentration of one (or more) species becomes negative, this could be due to the fact that the starting conditions are too far from equilibrium and/or $\delta c$ is too large. Can you think of how this problem can be fixed automatically in the implementation of these algorithm.


## Assignment #1
### Part a:  Dissociation of a mono-protic acid
Calculate the final pH of a solution of a 0.15M of acetic acid

\begin{equation}
\mathrm{CH_3COOH \rightleftharpoons CH_3COO^{-} + H^+} \qquad pK_{a} = 3.74
\end{equation}

Remember that the equilibrium constant for the reation
\begin{equation}
AH \leftrightharpoons A^- + H^+
\end{equation} 
is $K_{eq}=10^{-pK_a}$ and the reaction quotient
\begin{equation}
Q = \frac{[A^-][H^+]}{[HA]}
\end{equation}

Here below you can see the ICE table for this problem

|    <img width=50/> | [HA]<img width=150/> | [A$^-$]<img width=150/> | [H$^+$]<img width=150/>
| :---: |:---------: |:---------: |:---------:
| *I*  | [HA]$_0$ | [A$^-$]$_0$ | [H$^+$]$_0$
| *C*  | $-F_1\delta c$ | $+F_1\delta c$ | $+F_1\delta c$
| *E*  | [HA]$_0-F\delta c$ | [A$^-$]$_0+F_1\delta c$ | [H$^+$]$_0+F\delta c$


### Part b: External titration
Let's now imagine that the pH of the solution of the previous example is kept constant by external titration.

1. What do you have to change in your program to account for this?
2. What is the concentration of [AH] if the pH is kept to values of 2, 4 and 6?
3. Try fixing the pH at a value greater than 9, does the procedure work? (see critical thinking question #4)

## Assignment #2:  Coupled reactions
Let's now imagine we have two coupled reactions in the system, _e.g._ oxalic acid in pure water.

\begin{eqnarray}
H_2C_2O_4 + H_2O &\leftrightharpoons& H_3O^+ + HC_2O_4^- &\qquad\qquad& pK_{a1} = 1.25\\
HC_2O_4^- + H_2O &\leftrightharpoons& H_3O^+ + C_2O_4^= &\qquad\qquad& pK_{a2} = 4.14 \\
\end{eqnarray}

Determine 
1. What is the pH of 1 L solution prepared with 18 g of oxalic acid
2. What would be the equilibrium concentrations of $HC_2O_4$, $HC_2O_4^-$ and $C_2O_4^=$ if the pH of the same solution is maintained at a value of 3 using an extermal titration.

For brevity, let's call oxalic acid "H$_2$A".

You now have to compute two reaction quotients

\begin{eqnarray}
Q_1 = \frac{[HA^-][H^+]}{[H_2A]} &\qquad\qquad& Q_2 = \frac{[A^=][H^+]}{[HA^-]}\\
\end{eqnarray}

which will give rise to two forces, $F_1$ and $F_2$,

\begin{equation}
F_1 = RT \ln\bigg[\frac{K_{a1}}{Q_1}\bigg]
\end{equation}
\begin{equation}
F_2 = RT \ln\bigg[\frac{K_{a2}}{Q_2}\bigg]
\end{equation}

You can then use a "double" ICE table to update the concentrations.

|    <img width=50/>  | [H$_2$A]  <img width=150/> | [HA$^-$]<img width=150/> | [A$^=$]<img width=150/> | [H$^+$]<img width=150/>
| :---: | :--------: |:---------: |:---------: |:---------:
| *I*  | [H$_2$A]$_0$ | [HA$^-$]$_0$ | [A$^=$]$_0$ | [H$^+$]$_0$
| *C1*  | $-F_1\delta c$ | $+F_1\delta c$ | $-$ | $+F_1\delta c$
| *C2*  | $-$ | $-F_2\delta c$ | $+F_2\delta c$  | $+F_2\delta c$
| *E*  | [H$_2$A]$_0-F_1\delta c$ | [HA$^-$]$_0+(F_1-F_2)\delta c$ | [A$^=$]$_0+F_2\delta c$ | [H$^+$]$_0+(F_1+F_2)\delta c$


# Lab report

### Introduction:
1. Explain the working principle of this numerical procedure to solve chemical equilibrium problems
    1. Start from the definition of progress of a reaction and the change in free energy using the chemical potential
    2. define the driving force
    3. explain the procedure using a "pseudo code" or a flow chart

### Results:
1. Implement the numerical procedure in python/excel and solve the two assignment problems below

2. Show graphs of the concentrations of all the species _vs_ the number of cycles to show that your calculation is converged. In some cases a logarithmic scale would make the graph clearer.

3. Verify your numeric solutions by solving the problem analytically (show all the calculations).

### Critical thinking questions
Instead of the conclusions section, answer the "critical thinking" questions stated above.

### References
No references are needed for this lab.
